Import necessary libraries

In [ ]:
import numpy as np
from astropy.io import fits
import sep
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
from matplotlib import rcParams

Additional setup to display plots

In [ ]:
%matplotlib inline
rcParams['figure.figsize'] = [10., 8.]


Read the UDF f105w image from a FITS file

In [ ]:
udf_f105w_file_path = "hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits"
with fits.open(udf_f105w_file_path) as hdul:
    data_udf = hdul[0].data


Display the image and save to PNG

In [ ]:
m_udf, s_udf = np.mean(data_udf), np.std(data_udf)
plt.imshow(data_udf, interpolation='nearest', cmap='gray', vmin=m_udf-s_udf, vmax=m_udf+s_udf, origin='lower')
plt.colorbar()
plt.savefig('udf_image.png') 
plt.show()

Measure spatially varying background on the UDF image

In [ ]:
bkg_udf = sep.Background(data_udf)

Get "global" mean and noise of the image background

In [ ]:
print("Global background mean (UDF):", bkg_udf.globalback)
print("Global background noise (UDF):", bkg_udf.globalrms)

Evaluate background as 2-d array, same size as original UDF image and save as PNG

In [ ]:
bkg_image_udf = bkg_udf.back()
plt.imshow(bkg_image_udf, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar()
plt.savefig('udf_background.png')  
plt.show()

Evaluate the background noise as 2-d array and save as PNG

In [ ]:
bkg_rms_udf = bkg_udf.rms()
plt.imshow(bkg_rms_udf, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar()
plt.savefig('udf_background_noise.png')  
plt.show()

Subtract the Background from the UDF data

In [ ]:
data_sub_udf = data_udf - bkg_udf

Run object detection on the background-subtracted UDF data

In [ ]:
objects_udf = sep.extract(data_sub_udf, 1.5, err=bkg_udf.globalrms)
print("Number of objects detected (UDF):", len(objects_udf))

Plot background-subtracted UDF image with ellipses around detected objects

fig, ax = plt.subplots()
m_udf, s_udf = np.mean(data_sub_udf), np.std(data_sub_udf)
im_udf = ax.imshow(data_sub_udf, interpolation='nearest', cmap='gray', vmin=m_udf-s_udf, vmax=m_udf+s_udf, origin='lower')

for i in range(len(objects_udf)):
    e = Ellipse(xy=(objects_udf['x'][i], objects_udf['y'][i]),
                width=6*objects_udf['a'][i],
                height=6*objects_udf['b'][i],
                angle=objects_udf['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)

plt.savefig('udf_detected_obfigure as PNG
plt.show()NG
plt.show()

Perform circular aperture photometry with a 3 pixel radius on the UDF data

In [ ]:
flux_udf, fluxerr_udf, flag_udf = sep.sum_circle(data_sub_udf, objects_udf['x'], objects_udf['y'],
                                                 3.0, err=bkg_udf.globalrms, gain=1.0)

Display the first 10 objects' results

In [ ]:
for i in range(10):
    print(f"Object {i}: flux = {flux_udf[i]:.6f} +/- {fluxerr_udf[i]:.6f}")


Save the flux histogram as a figure

In [ ]:
plt.hist(flux_udf, bins=30, color='blue', alpha=0.7)
plt.xlabel('Flux (UDF)')
plt.ylabel('Frequency')
plt.savefig('udf_flux_histogram.png')  # Save the figure as PNG
plt.show()

In [ ]:
Calculate mean, median, and standard deviation of the flux distribution

In [ ]:
mean_flux_udf = np.mean(flux_udf)
median_flux_udf = np.median(flux_udf)
std_dev_flux_udf = np.std(flux_udf)

print(f"Mean flux (UDF): {mean_flux_udf:.6f}")
print(f"Median flux (UDF): {median_flux_udf:.6f}")
print(f"Standard deviation of flux (UDF): {std_dev_flux_udf:.6f}")

In [ ]:
Find the largest outlier in the distribution

In [ ]:
largest_outlier_idx_udf = np.argmax(np.abs(flux_udf - mean_flux_udf))
largest_outlier_flux_udf = flux_udf[largest_outlier_idx_udf]

print(f"Largest outlier in flux (UDF): {largest_outlier_flux_udf:.6f}")

Save the RGB 3-color false image of UDF
Download the f125w and f160w images of the HUDF and get their data
Assume f125w_image.fits and f160w_image.fits as the filenames

In [ ]:
f105w_file_path = "C:\Users\Sanket Dadhwal\Downloads\hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits"
f125w_file_path = "C:\Users\Sanket Dadhwal\Downloads\hlsp_hudf12_hst_wfc3ir_udfmain_f125w_v1.0_drz (1).fits"  
f160w_file_path = "C:\Users\Sanket Dadhwal\Downloads\hlsp_hudf12_hst_wfc3ir_udfmain_f160w_v1.0_drz.fits"  

with fits.open(f105w_file_path) as hdul_f105w:
    data_f125w = hdul_f105w[0].data

with fits.open(f125w_file_path) as hdul_f125w:
    data_f125w = hdul_f125w[0].data

with fits.open(f160w_file_path) as hdul_f160w:
    data_f160w = hdul_f160w[0].data

In [ ]:
Normalize data

In [ ]:
data_f105w = data_f105w / np.max(data_f105w)
data_f125w = data_f125w / np.max(data_f125w)
data_f160w = data_f160w / np.max(data_f160w)

Create the RGB image

In [ ]:
rgb_image = np.stack([data_f160w, data_f125w, data_f105w], axis=-1)

In [ ]:
Display the RGB image

In [ ]:
plt.imshow(rgb_image, origin='lower')
plt.axis('off')
plt.savefig('udf_rgb_image.png', bbox_inches='tight', pad_inches=0)  # Save as PNG
plt.show()